<a href="https://colab.research.google.com/github/KanthiKiranS/KK-Repo/blob/master/Pheme_Veracity_1__2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import fastai

In [2]:
from fastai.text import * 

In [3]:
def read_veracity(path_to_file, shuffle=False):
    def enc(col):
        if col['label'] == 2.0:
            return 'Unverified'
        elif col['label'] == 1.0:
            return 'True'
        elif col['label'] == 0.0:
            return 'False'

    df = pd.read_csv(path_to_file)
    df = df.drop(['Unnamed: 0', '0'], 1)
    df.columns = ['text', 'label']
    df['label'] = df.apply(enc, 1)
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    return df.dropna().iloc[:, ::-1]

In [4]:
# For veracity
train = read_veracity('modifiedTrain_New.csv', True)
test = read_veracity('modifiedTest_New.csv')

In [5]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [6]:
pd.concat([train['text'], test['text']]).reset_index(drop=True).to_csv('text.csv')

In [7]:
data_lm = TextLMDataBunch.from_csv('', 'text.csv')
data_clas = TextClasDataBunch.from_csv('', 'train.csv', vocab=data_lm.train_ds.vocab, text_cols='text', label_cols='label', bs=32)

In [8]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [9]:
data_lm = load_data('', 'data_lm_export.pkl')
data_clas = load_data('', 'data_clas_export.pkl', bs=16)

In [10]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.2)
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.452897,4.230325,0.296921,03:36


In [11]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.901798,3.927474,0.335312,04:12


In [12]:
learn.predict("This is a review about", n_words=10)

'This is a review about justified behavior of hostages , prepared to be " repeatedly'

In [13]:
learn.save_encoder('ft_enc')

In [14]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (61256 items)
x: TextList
xxbos @orenyh @thetimes @brookestimes para xxunk en armas y xxunk,xxbos @matthewkick @rolandsmartin,xxbos xxmaj over a thousand have gathered at xxmaj martin xxmaj place to pay their respects # charliehebdo # jesuischarlie . xxmaj photo : @shooterwol http : / / t.co / xxunk,xxbos @unpreachedtruth @wanderer19 xxmaj protecting and serving , right ?,xxbos xxup @rc5251 @danteb4u @fox2now @arrogantdemon xxmaj there also is xxmaj meth - xxmaj headed xxmaj xxunk on the xxmaj texas border pointing rifles at skinny little kids .
y: CategoryList
False,False,False,True,False
Path: .;

Valid: LabelList (15315 items)
x: TextList
xxbos @youranonnews xxup xxunk xxup computers xxup were xxup hacked xxrep 5 !,xxbos @cnnbrk fuck you xxup isis and you coward terrorist pricks,xxbos @suaye _ @jamalhbryant xxmaj check out xxmaj xxunk xxmaj for xxmaj the xxmaj best ( xxmaj official xxmaj video ) http : / / t.co / xxunk,xxbos @thea

In [15]:
data_clas.show_batch()

text,target
xxbos “ @wesleylowery : a few feet from where # michaelbrown was killed : # xxmaj ferguson http : / / t.co / xxunk ” \n “ @wesleylowery : a few feet from where # michaelbrown was killed : # xxmaj ferguson http : / / t.co / xxunk \n “ @wesleylowery : a few feet from where # michaelbrown was killed : # xxmaj ferguson http :,False
xxbos @bbcweather @bbcworld xxup welcom xxup to xxup the xxup city xxup of xxup the xxup shepherd xxup light xxup given xxup to xxup humanity xxup and xxup death xxup given xxup by xxup your xxup owe xxup xxunk xxup god xxup almighty xxup xxunk,False
xxbos @guardian xxup us & & xxup uk are so greedy they arm xxmaj terrorists just to xxmaj loot other xxmaj nations . xxup us has not learnt from 9 / 11 & & xxup us armed xxup obl / xxmaj taliban . xxmaj xxunk,False
"xxbos “ xxup @wsj : xxmaj paris , xxmaj london , xxmaj madrid , xxmaj san xxmaj francisco , xxmaj new xxmaj york . xxmaj video footage from # jesuischarlie gatherings around the world : http : / / t.co / xxunk",False
xxbos xxup rt @freetopher : xxup officer xxup darren xxup wilson xxup standing xxup over xxup mike xxup brown xxup 's xxup body xxup after xxup shooting xxup him 9 xxup times # xxmaj ferguson http : / / t.co / xxunk,False


In [16]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.517440,0.479755,0.768136,03:14


In [17]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.467205,0.406524,0.813386,03:26


In [18]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(2e-3/100, 2e-3), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.462981,0.383215,0.827294,07:28
1,0.417004,0.349420,0.838002,07:04
2,0.335168,0.301676,0.867711,07:13
3,0.314652,0.264304,0.884492,07:34
4,0.310167,0.250183,0.890630,07:40
5,0.232187,0.235590,0.895462,07:22
6,0.212882,0.231102,0.900490,07:08
7,0.187073,0.233820,0.900816,07:04
8,0.158477,0.241446,0.901600,07:39
9,0.145655,0.240113,0.901861,07:44


In [19]:
l = []
for text in test['text']:
    l.append(str(learn.predict(text)[0]))

In [20]:
from sklearn.metrics import accuracy_score, f1_score

In [21]:
accuracy_score(test['label'], l)

0.6806551091848642

In [22]:
f1_score(test['label'], l, average='macro')

0.6239943509920872

In [23]:
learn.export('veracity.pkl')